# Running dataset with basic convolutional model

If you want to see some basics about the data and the images, refer to the other notebooks

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torch.optim as optim

/opt/miniconda3/envs/basic-pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

import data_handling as data
import model

## Creating dataset

The code for this dataset is included in ```data_handling.py```. 

In [3]:
needle_directory = '/Users/carlosolivares/be224-fp/data/NeedleImages/'
labels_path = os.path.join(needle_directory, 'Labels.csv')

data_transformer = transforms.Compose([transforms.ToTensor()])

dataset = data.NeedleImageDataset(
    path2data=needle_directory,
    path2labels=labels_path,
    transform = data_transformer
)

In [4]:
len_dataset = len(dataset)
len_train = int(0.8 * len_dataset)
len_val = len_dataset - len_train

train_ds, val_ds = random_split(dataset, [len_train, len_val])

print("train datset length:", len(train_ds))
print("validation dataset length:", len(val_ds))

train_dl = DataLoader(train_ds, batch_size=10, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=10, shuffle=False)

train datset length: 504
validation dataset length: 127


In [5]:
## checking the length of the dataloaders
print(len(train_dl))

51


In [6]:
"""
checking to see that the batches come appropriately shaped for input to model. 
The model accepts data in (batch, channels, height, width)
Our images are grayscale, so we stack the values to get 3 channels like an RGB
"""

for xb, yb in val_dl:
    print(xb.shape)
    print(yb.shape)
    break

torch.Size([10, 3, 512, 512])
torch.Size([10])


## Model option 1

This one has two output nodes with no output activation and ```torch.nn.CrossEntropyLoss``` as the objective function.

In [7]:
naivenet = model.NaiveNet()

### Training the model for ten epoch to see if it actually learns



In [8]:
weights = [.3027, .6973]
criterion = torch.nn.CrossEntropyLoss(reduction='mean', weight=torch.tensor(weights))
optimizer = optim.Adam(naivenet.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-03.


In [9]:
def train_on_epoch(model, dataloader, loss_fn, optimizer):
    """
    """
    epoch_loss = 0
    running_loss = 0
    for i, batch in enumerate(dataloader):
        inputs, labels = batch
        new_loss = train_on_minibatch(
            model = model,
            inputs = inputs, 
            labels = labels,
            loss_fn = loss_fn,
            optimizer = optimizer
        )
        running_loss += new_loss
        epoch_loss += new_loss
        if i % 10 == 9:    # print every 10 mini-batches
            print('Mean of loss reduction on minibatch {}: {}'.format(i+1, running_loss / 10.0))
            #print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0
    mean_loss = epoch_loss / float(len(dataloader))
    return mean_loss


def train_on_minibatch(model, inputs, labels, loss_fn, optimizer):
    """
    """
    model.train()
    outputs = model(inputs)
    loss = loss_fn(outputs, labels.to(torch.int64))
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(4):
    mean_loss_on_epoch = model.train_on_epoch(
        model = naivenet,
        dataloader = train_dl,
        loss_fn = criterion,
        optimizer = optimizer
    )
    print('Mean loss on epoch {}: {}'.format((epoch+1), mean_loss_on_epoch))
    val_loss = model.get_validation_loss(naivenet, criterion, val_dl)
    print('validation loss after epoch {}: {}'.format((epoch+1), val_loss))
    mean_accuracy = model.get_accuracy_on_dataloader(
        model = naivenet,
        dataloader = val_dl,
        activation = torch.nn.Softmax(dim=1)
    )
    print('mean accuracy on validation set after epoch {}: {}'.format(epoch+1, mean_accuracy))
    scheduler.step()

print('Finished Training')

Mean of loss reduction on minibatch 10: 0.626650920510292
Mean of loss reduction on minibatch 20: 0.5935717463493347
Mean of loss reduction on minibatch 30: 0.6448981046676636
Mean of loss reduction on minibatch 40: 0.7058503448963165
Mean of loss reduction on minibatch 50: 0.9383671462535859
Mean loss on epoch 1: 0.7068986454430748
validation loss after epoch 1: 0.08548550296017504
mean accuracy on validation set after epoch 1: 0.4131868131868131
Adjusting learning rate of group 0 to 1.0000e-03.
Mean of loss reduction on minibatch 10: 0.6949649810791015
Mean of loss reduction on minibatch 20: 0.5787642538547516
Mean of loss reduction on minibatch 30: 0.6483381152153015
Mean of loss reduction on minibatch 40: 0.6382522404193878
Mean of loss reduction on minibatch 50: 0.6466220319271088
Mean loss on epoch 2: 0.641073514433468
validation loss after epoch 2: 0.06507388484759594
mean accuracy on validation set after epoch 2: 0.6714285714285714
Adjusting learning rate of group 0 to 1.0000e-

In [10]:
## how do we get an accuracy score? answer here

for batch in val_dl:
    inputs, labels = batch
    outputs = naivenet(inputs)
    break

In [25]:
torch.nn.Softmax()(outputs)

/opt/miniconda3/envs/basic-pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.4696, 0.5304],
        [0.4512, 0.5488],
        [0.4488, 0.5512],
        [0.4491, 0.5509],
        [0.4433, 0.5567],
        [0.4499, 0.5501],
        [0.4697, 0.5303],
        [0.4554, 0.5446],
        [0.4429, 0.5571],
        [0.4467, 0.5533]], grad_fn=<SoftmaxBackward>)

In [29]:
probs = torch.nn.Softmax(dim=1)(outputs)
print(probs.shape)
classes = torch.argmax(probs, 1)
classes

torch.Size([10, 2])


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [26]:
a = torch.randn(4, 2)
a = torch.nn.Softmax(dim=1)(a)
print(a)
print(torch.argmax(a, dim=1))
labels = torch.Tensor([1, 1, 1, 0])
print(torch.argmax(a, dim=1) == labels.to(torch.int64))
print(sum(torch.argmax(a, dim=1) == labels.to(torch.int64)).item())
print(a.shape[0])

tensor([[0.4729, 0.5271],
        [0.3009, 0.6991],
        [0.7995, 0.2005],
        [0.1875, 0.8125]])
tensor([1, 1, 0, 1])
tensor([ True,  True, False, False])
2
4


In [15]:
print(labels)

tensor([1., 1., 1., 0.])


In [27]:
def get_accuracy(preds, labels, activation):
    """
    """
    probs = activation(preds)
    classes = torch.argmax(probs, dim=1)
    num_correct = sum(classes == labels).item()
    num_examples = preds.shape[0]
    return num_correct / float(num_examples)

a = torch.randn(4, 2)
a = torch.nn.Softmax(dim=1)(a)
print(a)
print(torch.argmax(a, dim=1))
labels = torch.Tensor([1, 1, 1, 0])
print(get_accuracy(a, labels, torch.nn.Softmax(dim=1)))   
    

tensor([[0.2826, 0.7174],
        [0.1433, 0.8567],
        [0.6965, 0.3035],
        [0.5780, 0.4220]])
tensor([1, 1, 0, 0])
0.75


In [20]:
labels

tensor([1., 0., 1., 1., 0., 0., 1., 1., 1., 1.], dtype=torch.float64)

In [ ]:
## investigating how the crossentrupy loss works. From the docs: 

# Example of target with class indices
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
#output.backward()
# Example of target with class probabilities
#input = torch.randn(3, 5, requires_grad=True)
#target = torch.randn(3, 5).softmax(dim=1)
#output = loss(input, target)
#output.backward()

### Looking at how the loss function works

In [14]:
input

tensor([[-0.6759, -0.8502,  0.7985, -0.5464,  1.2984],
        [-1.1789, -1.6076, -0.6740,  1.3447, -0.6631],
        [-1.3812,  0.5992,  2.3664,  0.8805,  0.3675]], requires_grad=True)

In [15]:
target

tensor([1, 0, 2])

In [16]:
output

tensor(2.0512, grad_fn=<NllLossBackward>)

In [17]:
outputs

tensor([[-0.4508,  0.3201],
        [-0.4884,  0.3111],
        [-0.4619,  0.3550],
        [-0.5006,  0.3499]], grad_fn=<AddmmBackward>)

In [18]:
labels

tensor([0, 0, 1, 1])

In [19]:
loss(outputs, labels)

tensor(0.7609, grad_fn=<NllLossBackward>)

In [20]:
len(val_dl.dataset)

127

In [ ]:
# the next thing to do is actually get the metrics from model and stuff. 
# looks like you might need an lr schedulaer

## Model option 2

Same model but with one output node, sigmoid activation, and ```torch.nn.BCELoss```. 